In [ ]:
# acid enumeration #
## set up reactants ## 
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools as pdt, Draw
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from tqdm import tqdm
import numpy as np
from rdkit.Chem.SaltRemover import SaltRemover

## SMARTS for alkyl halide ##

carbox_acid = Chem.MolFromSmarts('[#8:9][#6:1]=[#8:10]')

## prepare selection of molecules ##

scaffold_sdf = Chem.SDMolSupplier('c:/Users/sgjhewi2/480 Enumeration/12a.sdf')
scaffold = [Chem.AddHs(x) for x in scaffold_sdf]  # adds Hs for the scaffold molecule

suppl = Chem.MultithreadedSDMolSupplier('c:/Users/sgjhewi2/480 Enumeration/Libraries/Enamine_CarboxylicAcids_43649cmps_20240202.sdf', numWriterThreads=10, removeHs=False)
mols = [x for x in suppl if x is not None]  # compiles all molecules in the library file

reactants = []
for i in mols:
    try:
        i.RemoveAllConformers()     # removes al conformations on the molecule so we can modify it
        reactants.append(i)
    except:
        continue

acids = [x for x in reactants if x.HasSubstructMatch(carbox_acid)]  # the molecule is kept if it contains a substructure match to the acid
non_match = [x for x in reactants if not x.HasSubstructMatch(carbox_acid)] # molecule added to non-matches list if not matched to substructure

remover = SaltRemover() # any salts present are removed
acid_nosalt = []
for x in acids:
    x = remover.StripMol(x)
    acid_nosalt.append(x)

print('No. Molecules in file:', len(reactants))
print('Acid Matches:', len(acids))
print('Non-matches:', len(non_match))

In [29]:
# reaction enumeration # 
mols = [] # list for our enumerated products

rxn = AllChem.ReactionFromSmarts('[#8:10][#6:1]=[#8:9].[#1]-[#7:8]-[#6:2]1:[#6:3]:[#6:4]:[#6:5]:[#6:6]:[#6:7]:1>>[#8:9]=[#6:1]-[#7:8]-[#6:2]1:[#6:3]:[#6:4]:[#6:5]:[#6:6]:[#6:7]:1')
prods = AllChem.EnumerateLibraryFromReaction(rxn, [acids, scaffold]) # the reaction between the acid and the scaffold amine

for i,x in enumerate(prods):
    mols.append(x)

mols_2 = []
for a in mols:
    for x in a:
        mols_2.append(x)

df = pd.DataFrame(mols_2, columns=['Molecule']) # adds the molecule object to a dataframe (table)

prods_smiles = []
for i,row in df.iterrows():
    try:
       smi = Chem.MolToSmiles(Chem.RemoveHs(row.Molecule)) # for each molecule, convert it to a SMILES string
       prods_smiles.append(smi)
    except:
        prods_smiles.append(None)
df['SMILES'] = prods_smiles

print('Products:', len(df)/4) # divided by 4 as the acid matching matches twice due to matching to one orientation and then a different orientation of the same group

Products: 53076.0


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

a = Chem.MolFromSmarts('[#8:9][#6:1]=[#8:10]')
b = Chem.MolFromSmarts('[#1]-[#7:8]-[#6:2]1:[#6:3]:[#6:4]:[#6:5]:[#6:6]:[#6:7]:1')
c = Chem.MolFromSmarts('[#8:9]=[#6:1]-[#7:8]-[#6:2]1:[#6:3]:[#6:4]:[#6:5]:[#6:6]:[#6:7]:1')

display(Draw.MolToImage(a))
display(Draw.MolToImage(b))
display(Draw.MolToImage(c))


In [ ]:
## initialise brenk filter ##

params = FilterCatalogParams()
params.AddCatalog(FilterCatalogParams.FilterCatalogs.BRENK)  #  a catalog for the Brenk structural filters
brenk_catalog = FilterCatalog(params)

## search for brenk matches ##

# tqdm(df.iterrows(), total=df.shape[0]):
matches = []
clean = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        molecule = row.Molecule
        Chem.SanitizeMol(molecule) # sanitize the molecule 
        entry = brenk_catalog.GetFirstMatch(molecule)  # Get the first matching BRENK
    except:
        continue
    if entry is not None:
        # store BRENK information
        matches.append(
            {
                "SMILES": row.SMILES,
                "rdkit_molecule": row.Molecule,
                "Brenk": entry.GetDescription().capitalize(),
            }
        )
    else:
        # collect indices of molecules without BRENK
        clean.append(index)

matches = pd.DataFrame(matches) # a dataframe of all the Brenk matches
no_brenk = df.loc[clean]  # keep molecules without BRENK

print(f"Number of compounds with BRENK: {len(matches)/4}")
print(f"Number of compounds without BRENK: {len(no_brenk)/4}")

no_brenk_mols = no_brenk[no_brenk.index % 4 == 0]
matched_mols = matches[matches.index % 4 == 0]

no_brenk_mols = no_brenk.reset_index(drop=True) # end up with a dataframe of molecules that have no structural alerts
matched_mols = matches.reset_index(drop=True) # end up with a dataframe of molecules that have structural alerts

In [63]:
# lipinski and csv writers # 
# each section here should be run separately
from esol import ESOLCalculator
from rdkit.Chem import Descriptors, QED, Lipinski
import pandas as pd

ESOLCalculator = ESOLCalculator()

def lipinski_filter(df):
    filtered_df = df.copy()
    #filtered_df = filtered_df[filtered_df['MW'] <= 600]
    filtered_df = filtered_df[filtered_df['HBA'] <= 10]
    filtered_df = filtered_df[filtered_df['HBD'] <= 5]
    filtered_df = filtered_df[filtered_df['ALOGP'] <= 5]
    filtered_df = filtered_df[filtered_df['ROTB'] <= 10]
    return filtered_df 

# ------------------------------mols that pass brenk ---------------------------------- # 

props = []
for mol in no_brenk_mols.Molecule:
    props.append(list(QED.properties(mol))) # quantitative estimate of drug-likeness (QED)

hvyatoms = [x.GetNumHeavyAtoms() for x in no_brenk_mols.Molecule] # calculates number of heavy atoms
esol = [ESOLCalculator.calc_esol(x) for x in no_brenk_mols.Molecule] # predicts solubility

brenk_pass = pd.DataFrame(props, columns=('MW','ALOGP','HBA','HBD','PSA','ROTB','AROM','ALERTS')) # adds all QED properties to table

brenk_pass.reset_index(drop=True)
brenk_pass.insert(0, 'MOL', no_brenk_mols.Molecule)
brenk_pass.insert(8, 'LogS', esol)
brenk_pass.insert(9, 'HVYATOMS', hvyatoms)  # adds extra columns for other bits of info
brenk_pass.insert(10, 'SMILES', no_brenk_mols.SMILES)

brenk_df = lipinski_filter(brenk_pass) # df filtered by Lipinski Ro5 
brenk_df = brenk_df[brenk_df.index % 4 == 0]
brenk_lip_df = brenk_df.reset_index(drop=True)
brenk_lip_df.to_csv('C:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_pass (lipinski - no MWcutoff).csv') # saves data table for csv / xl file

# ----------------------------------------- mols that fail brenk ---------------------------- # 

brenk_fail_mols = matched_mols.reset_index(drop=True)

props2 = []
for mol in matched_mols.rdkit_molecule:
    props2.append(list(QED.properties(mol)))

hvyatoms2 = [x.GetNumHeavyAtoms() for x in matched_mols.rdkit_molecule]
esol2 = [ESOLCalculator.calc_esol(x) for x in matched_mols.rdkit_molecule]

brenk_fail = pd.DataFrame(props2, columns=('MW','ALOGP','HBA','HBD','PSA','ROTB','AROM','ALERTS')) # same as above but for mols with structural alerts

brenk_fail.reset_index(drop=True)
brenk_fail.insert(0, 'MOL', brenk_fail_mols.rdkit_molecule)
brenk_fail.insert(8, 'LogS', esol2)
brenk_fail.insert(9, 'HVYATOMS', hvyatoms2)
brenk_fail.insert(10, 'BRENK_FLAG', brenk_fail_mols.Brenk)
brenk_fail.insert(11, 'SMILES', brenk_fail_mols.SMILES)

brenk_fail = brenk_fail[brenk_fail.index % 4 == 0]
brenk_fail_df = brenk_fail.reset_index(drop=True)
brenk_fail_df.to_csv('c:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_fail.csv')

#-------------------------- whole dataframe regardless of pass/fail along with properties etc ------------------------------ #

props3 = []
for mol in df.Molecule:
    try:
        props3.append(list(QED.properties(mol)))
    except:
        continue

whole_df = pd.DataFrame(props3, columns=('MW','ALOGP','HBA','HBD','PSA','ROTB','AROM','ALERTS')) # same as above but for all molecules, Brenk or not
whole_df.insert(0, 'MOL', df.Molecule)

hvyatoms3 = []
for x in whole_df.MOL:
    try:
        a = x.GetNumHeavyAtoms()
        hvyatoms3.append(a)
    except:
        continue

esol3 = []
for i in whole_df.MOL:
    try:
        b = ESOLCalculator.calc_esol(i)
        esol3.append(b)
    except:
        continue

whole_df.insert(8, 'LogS', esol3)
whole_df.insert(9, 'HVYATOMS', hvyatoms3)
whole_df.insert(10, 'BRENK_FLAG', brenk_fail_mols.Brenk)
whole_df.insert(11, 'SMILES', df.SMILES)

whole_df = whole_df[whole_df.index % 4 == 0]  
all_mol_df = whole_df.reset_index(drop=True)
all_mol_df.to_csv('c:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_All.csv')

In [119]:
structures = []
for x in brenk_lip_df.MOL:
    AllChem.EmbedMolecule(x) 
    Chem.AddHs(x)
    try:
        AllChem.MMFFOptimizeMolecule(x)  # for each molecule in the filtered and non-alert dataframe, energy minimise and write to an sdf file
    except: 
        continue
    structures.append(x)

writer = Chem.SDWriter('C:/Users/sgjhewi2/480 Enumeration/SDF Files/Filtered_CA/Filtered_CA_Library.sdf')

for i in structures:
    writer.write(i)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

properties = pd.read_csv('c:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_pass (lipinski - 650MWcutoff).csv')

data = pd.DataFrame(properties)

mw = data.MW
logS = data.LogS
logP = data.ALOGP
rotbonds = data.ROTB


print(len(data))

plt.figure(figsize=(10,10))
plt.subplot(221)
plt.hist(mw, bins=10, edgecolor='black', color='lightblue')
plt.xticks(np.arange(400,850,50))
plt.xlim(450,800)
plt.title('Molecular Weight')

plt.subplot(222)
plt.hist(logS, bins=10, edgecolor='black', color='lightblue') 
plt.xticks(np.arange(-10,12,2))
plt.xlim(-10,10)
plt.title('LogS')

plt.subplot(223)
plt.hist(logP, bins=10, edgecolor='black', color='lightblue')
plt.xticks(np.arange(0,10,2))
plt.xlim(0,8)
plt.title('LogP')

plt.subplot(224)
plt.hist(rotbonds, bins=10, edgecolor='black', color='lightblue')
plt.xticks(np.arange(0,12,2))
plt.xlim(0,10)
plt.title('RotBonds')
plt.suptitle('Product Properties from Carboxylic Acid Enumeration (Post-Brenk/Lipinski Filters with MW650 cut-off)')

In [65]:
# lipinski and csv writers # 
from esol import ESOLCalculator
from rdkit.Chem import Descriptors, QED, Lipinski
import pandas as pd

ESOLCalculator = ESOLCalculator()

def lipinski_filter(df):
    filtered_df = df.copy()
    filtered_df = filtered_df[filtered_df['MW'] <= 650]
    filtered_df = filtered_df[filtered_df['HBA'] <= 10]
    filtered_df = filtered_df[filtered_df['HBD'] <= 5]
    filtered_df = filtered_df[filtered_df['ALOGP'] <= 5]
    filtered_df = filtered_df[filtered_df['ROTB'] <= 10]
    return filtered_df 

# ------------------------------mols that pass brenk ---------------------------------- # 

props = []
for mol in no_brenk_mols.Molecule:
    props.append(list(QED.properties(mol))) # quantitative estimate of drug-likeness

hvyatoms = [x.GetNumHeavyAtoms() for x in no_brenk_mols.Molecule]
esol = [ESOLCalculator.calc_esol(x) for x in no_brenk_mols.Molecule]

brenk_pass = pd.DataFrame(props, columns=('MW','ALOGP','HBA','HBD','PSA','ROTB','AROM','ALERTS'))

brenk_pass.reset_index(drop=True)
brenk_pass.insert(0, 'MOL', no_brenk_mols.Molecule)
brenk_pass.insert(8, 'LogS', esol)
brenk_pass.insert(9, 'HVYATOMS', hvyatoms)
brenk_pass.insert(10, 'SMILES', no_brenk_mols.SMILES)

brenk_df = lipinski_filter(brenk_pass) # df filtered by Lipinski Ro5 
brenk_df = brenk_df[brenk_df.index % 4 == 0]
brenk_lip_df = brenk_df.reset_index(drop=True)
brenk_lip_df.to_csv('C:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_pass (lipinski - 650MWcutoff).csv')

In [148]:
from rdkit import Chem
from rdkit.Chem import Draw, QED
from esol import ESOLCalculator
import pandas as pd

scaffold = Chem.SDMolSupplier('c:/Users/sgjhewi2/480 Enumeration/12a.sdf')
mol = [x for x in scaffold]

ESOLCalculator = ESOLCalculator()

parent_props = []
for i in mol:
    parent_props.append(list((QED.properties(i))))

parent_df = pd.DataFrame(parent_props, columns=('MW','ALOGP','HBA','HBD','PSA','ROTB','AROM','ALERTS'))

esol = [ESOLCalculator.calc_esol(x) for x in mol]
parent_df.insert(8, 'LogS', esol)
parent_df

,MW,ALOGP,HBA,HBD,PSA,ROTB,AROM,ALERTS,LogS
0,432.322,2.4844,4,3,90.7,5,2,1,-4.612918


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

properties = pd.read_csv('c:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_pass (lipinski - 650MWcutoff).csv')

data = pd.DataFrame(properties)

mw = data.MW
logS = data.LogS
logP = data.ALOGP
rotbonds = data.ROTB

p_mw = parent_df.MW
p_logS = parent_df.LogS
p_logP = parent_df.ALOGP
p_rotb = parent_df.ROTB


print(len(data))

plt.figure(figsize=(10,10))
plt.subplot(221)
plt.hist(mw, bins=10, edgecolor='black', color='lightblue')
plt.bar(p_mw, height=5000, width=4, color='green', alpha=0.5)
plt.xticks(np.arange(400,950,50))
plt.xlim(400,900)
plt.title('Molecular Weight')

plt.subplot(222)
plt.hist(logS, bins=10, edgecolor='black', color='lightblue')
plt.bar(p_logS, height=5000, width=0.3, color='green', alpha=0.5)
plt.xticks(np.arange(-10,12,2))
plt.xlim(-10,10)
plt.title('LogS')

plt.subplot(223)
plt.hist(logP, bins=10, edgecolor='black', color='lightblue')
plt.bar(p_logP, height=5000, width=0.1, color='green', alpha=0.5)
plt.xticks(np.arange(0,10,2))
plt.xlim(0,8)
plt.title('LogP')

plt.subplot(224)
plt.hist(rotbonds, bins=10, edgecolor='black', color='lightblue')
plt.bar(p_rotb, height=7000, width=0.2, color='green', alpha=0.5)
plt.xticks(np.arange(0,12,2))
plt.xlim(0,10)
plt.title('RotBonds')
plt.suptitle('Product Properties from Carboxylic Acid Enumeration \n(Post-Brenk/Lipinski Filters with MW650 cut-off) - Parent Scaffold (green)')

In [ ]:
from paretoset import paretoset
# pareto optimisation #
csv = pd.read_csv('c:/Users/sgjhewi2/480 Enumeration/CSV Files/ACIDS/Acids_pass (lipinski - 650MWcutoff).csv')
csv = csv.drop(columns=['Unnamed: 0','MOL','HBA','HBD','PSA','AROM','HVYATOMS','SMILES', 'ALERTS']) # remaining properties are MW, LogS, LogP and ROTB

pareto = paretoset(csv, sense=['min','min','min','max'])
opt_pareto = csv[pareto]

index_list = list(opt_pareto.index.values)

# ================ Pareto Molecules ================= # 

df2 = brenk_lip_df.iloc[index_list]
display(df2)
print('Solutions:', len(index_list))

imgs = []
for i in df2.SMILES:
    mol = Chem.MolFromSmiles(i)
    imgs.append(mol)

gridimage = Draw.MolsToGridImage(imgs, molsPerRow=4, subImgSize=(400,400))
display(gridimage)


In [ ]:
## ionisation / pH ##
# code to protonate free amins and deprotonate acids
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
import pandas as pd

sdf = Chem.SDMolSupplier('c:/Users/sgjhewi2/480 Enumeration/SDF Files/Pareto_CA/Pareto_CA_Library.sdf', removeHs=False)

sdf_mols = []
for y in sdf:
    y.RemoveAllConformers
    sdf_mols.append(y)

df = pd.DataFrame(sdf_mols)

df3 = df.copy()

def set_to_neutral_pH(mol: Chem):

    """
    Not great, but does the job.
    * Protonates amines, but not aromatic bound amines.
    * Deprotonates carboxylic acid, phosphoric acid and sulfuric acid, without ruining esters.
    """

    protons_added = 0
    protons_removed = 0

    for indices in mol.GetSubstructMatches(Chem.MolFromSmarts('[N;D1]')):
        atom = mol.GetAtomWithIdx(indices[0])
        if atom.GetNeighbors()[0].GetIsAromatic(): # determines the atom environment for a N atom
            continue # aniline

        atom.SetFormalCharge(1)
        protons_added += 1

    for indices in mol.GetSubstructMatches(Chem.MolFromSmarts('C(=O)[O;D1]')):
        atom = mol.GetAtomWithIdx(indices[2])
        # benzoic acid pKa is low. # determines the environment for the carboxylic acid group
        atom.SetFormalCharge(-1)
        protons_removed += 1

    for indices in mol.GetSubstructMatches(Chem.MolFromSmarts('P(=O)[Oh1]')):
        atom = mol.GetAtomWithIdx(indices[2])
        # benzoic acid pKa is low. # same for phosphoric and sulfuric
        atom.SetFormalCharge(-1)
        protons_removed += 1

    for indices in mol.GetSubstructMatches(Chem.MolFromSmarts('S(=O)(=O)[Oh1]')):
        atom = mol.GetAtomWithIdx(indices[3])
        # benzoic acid pKa is low.
        atom.SetFormalCharge(-1)
        protons_removed += 1

    return (protons_added, protons_removed)

for x in df3[0]:
    set_to_neutral_pH(x)  # for each molecule, run the function to protonate/deprotonate groups at neutral pH

mols = [x for x in df3[0]]


ions = []
for x in mols:
    AllChem.EmbedMolecule(x) 
    Chem.AddHs(x)
    AllChem.MMFFOptimizeMolecule(x) # energy minimise and write mols to sdf
    ions.append(x)

writer = Chem.SDWriter('C:/Users/sgjhewi2/480 Enumeration/SDF Files/Pareto_CA/Pareto_CA_Library_Ions.sdf')

for i in ions:
    writer.write(i)


In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(221)
plt.scatter(csv['MW'], csv['ALOGP'], label='All', zorder=10, s=2, alpha=0.8)
plt.xlabel('MW')
plt.ylabel('LogP')
plt.scatter(opt_pareto['MW'], opt_pareto['ALOGP'], label='Pareto', zorder=5, s=20, alpha=1)
plt.legend()

plt.subplot(222)
plt.scatter(csv['MW'], csv['LogS'], label='All', zorder=10, s=2, alpha=0.8)
plt.xlabel('MW')
plt.ylabel('LogS')
plt.scatter(opt_pareto['MW'], opt_pareto['LogS'], label='Pareto', zorder=5, s=20, alpha=1)
plt.legend()

plt.subplot(223)
plt.scatter(csv['MW'], csv['ROTB'], label='All', zorder=10, s=2, alpha=0.8)
plt.xlabel('MW')
plt.ylabel('RotBonds')
plt.scatter(opt_pareto['MW'], opt_pareto['ROTB'], label='Pareto', zorder=5, s=20, alpha=1)
plt.legend()

plt.subplot(224)
plt.scatter(csv['LogS'], csv['ALOGP'], label='All', zorder=10, s=2, alpha=0.8)
plt.xlabel('LogS')
plt.ylabel('LogP')
plt.scatter(opt_pareto['LogS'], opt_pareto['ALOGP'], label='Pareto', zorder=5, s=20, alpha=1)
plt.legend()